## Первая модель на vw, используем данный организаторами csv файл

In [20]:
from vw_sklearn import VWLogRegBinary
from sklearn.metrics import log_loss
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.utils import shuffle
import pandas as pd

pd.set_option("display.max_columns", 150)

In [21]:
all_data = pd.concat([pd.read_csv('./features.csv.zip',compression='zip'),\
                     pd.read_csv('./features_test.csv.zip',compression='zip')])

In [22]:
all_data.head()

,barracks_status_dire,barracks_status_radiant,d1_deaths,d1_gold,d1_hero,d1_items,d1_kills,d1_level,d1_lh,d1_xp,d2_deaths,d2_gold,d2_hero,d2_items,d2_kills,d2_level,d2_lh,d2_xp,d3_deaths,d3_gold,d3_hero,d3_items,d3_kills,d3_level,d3_lh,d3_xp,d4_deaths,d4_gold,d4_hero,d4_items,d4_kills,d4_level,d4_lh,d4_xp,d5_deaths,d5_gold,d5_hero,d5_items,d5_kills,d5_level,d5_lh,d5_xp,dire_boots_count,dire_bottle_time,dire_courier_time,dire_first_ward_time,dire_flying_courier_time,dire_tpscroll_count,dire_ward_observer_count,dire_ward_sentry_count,duration,first_blood_player1,first_blood_player2,first_blood_team,first_blood_time,lobby_type,match_id,r1_deaths,r1_gold,r1_hero,r1_items,r1_kills,r1_level,r1_lh,r1_xp,r2_deaths,r2_gold,r2_hero,r2_items,r2_kills,r2_level,r2_lh,r2_xp,r3_deaths,r3_gold,r3_hero,r3_items,r3_kills,r3_level,r3_lh,r3_xp,r4_deaths,r4_gold,r4_hero,r4_items,r4_kills,r4_level,r4_lh,r4_xp,r5_deaths,r5_gold,r5_hero,r5_items,r5_kills,r5_level,r5_lh,r5_xp,radiant_boots_count,radiant_bottle_time,radiant_courier_time,radiant_first_ward_time,radiant_flying_courier_time,radiant_tpscroll_count,radiant_ward_observer_count,radiant_ward_sentry_count,radiant_win,start_time,tower_status_dire,tower_status_radiant
0,0.0,51.0,0,996,4,6,0,3,12,1058,0,986,42,4,0,4,12,1085,0,1536,21,6,0,5,23,2052,0,500,37,8,0,3,2,742,0,1003,84,9,1,3,3,958,4,103.0,-84.0,-52.0,221.0,3,2,2,2874.0,9.0,NaN,1.0,7.0,7,0,0,1489,11,7,0,5,20,2098,0,991,67,4,0,3,10,842,0,1143,29,8,0,5,10,1909,0,741,20,7,0,3,6,757,1,658,105,11,0,3,4,732,2,134.0,-80.0,35.0,244.0,2,2,0,1.0,1430198770,0.0,1796.0
1,1.0,63.0,0,1384,39,8,0,5,16,1960,1,566,88,5,0,3,1,640,0,1350,79,12,2,3,2,720,0,583,7,7,0,2,0,440,0,1622,12,9,0,4,24,1470,4,149.0,-84.0,-5.0,195.0,5,3,1,2463.0,7.0,NaN,1.0,54.0,0,1,1,1033,42,12,0,4,9,1188,1,993,49,7,0,4,10,1596,0,1502,67,7,1,4,18,1506,0,631,37,7,0,3,7,669,0,539,26,5,0,2,1,415,0,173.0,-80.0,-20.0,NaN,2,2,0,1.0,1430220345,0.0,1974.0
2,63.0,0.0,1,2028,22,10,1,5,19,2305,1,959,66,10,0,3,19,1024,0,620,86,8,0,3,3,755,0,667,29,7,0,4,4,1319,0,1512,80,7,0,3,25,1350,4,45.0,-77.0,13.0,221.0,3,3,1,2130.0,3.0,NaN,0.0,224.0,7,2,0,1270,33,12,0,4,22,1319,0,775,98,6,0,3,6,1314,0,909,20,6,1,3,0,1297,1,2096,27,6,1,5,26,2360,0,1627,4,9,0,3,27,1395,5,63.0,-82.0,-39.0,NaN,2,2,1,0.0,1430227081,1830.0,0.0
3,63.0,50.0,0,1174,96,6,0,5,17,1878,0,1468,48,10,0,3,22,732,0,1051,15,7,0,4,11,1681,0,537,102,7,0,2,1,674,0,499,20,7,0,2,0,510,4,124.0,-80.0,27.0,184.0,0,2,0,1459.0,NaN,NaN,NaN,NaN,1,3,0,1056,29,5,0,4,14,1779,0,539,30,6,0,2,1,539,0,1139,75,6,0,5,15,2037,0,499,37,6,0,2,0,591,0,1075,41,6,0,3,12,712,3,208.0,-75.0,-30.0,NaN,0,2,0,0.0,1430263531,2047.0,1920.0
4,63.0,3.0,2,586,26,9,0,3,1,704,0,1665,69,7,1,3,20,1169,0,638,22,9,0,3,1,1055,0,1275,25,8,0,5,18,1815,1,904,8,7,0,4,6,1119,3,182.0,-80.0,-16.0,225.0,6,3,0,2449.0,6.0,NaN,1.0,-21.0,7,4,0,1090,13,8,1,4,8,1431,1,552,27,7,0,2,0,629,0,927,30,8,1,3,0,884,0,1439,72,11,1,3,16,925,0,880,93,8,0,4,7,1482,4,166.0,-81.0,46.0,181.0,1,2,0,0.0,1430282290,1974.0,4.0


In [46]:
def create_hl_word_bag(s):
    splitted = s.split('_')
    added_vals = [str(i)+'_'+splitted[1]+'_'+splitted[2] for i in range(int(splitted[0]))]
    added_vals.append(s)
    return ' '.join(added_vals)

In [47]:
all_data.first_blood_team = all_data.first_blood_team.replace(1.0,'r')
all_data.first_blood_team = all_data.first_blood_team.replace(0.0,'d')
all_data.first_blood_team = all_data.first_blood_team.fillna('no')

all_data.first_blood_player2 = all_data.first_blood_player2.fillna(0.0)
all_data.first_blood_player1 = all_data.first_blood_player1.fillna(0.0)
all_data.first_blood_player1 = all_data.first_blood_player1.astype(int)
all_data.first_blood_player2 = all_data.first_blood_player2.astype(int)

all_data.lobby_type = all_data.lobby_type.replace(7,'Ranked')
all_data.lobby_type = all_data.lobby_type.replace(0,'PublicMatchmaking')
all_data.lobby_type = all_data.lobby_type.replace(1,'Practice')

for feat in list(all_data.columns):
    if feat.endswith('level'):
        prefix = feat[:3]
        all_data[prefix+'levhero'] = all_data[feat].astype(str).str.cat(all_data[prefix+'hero'].astype(str),sep='_'+prefix[0]+'_')
    
for feat in list(all_data.columns):
    if feat.endswith('levhero'):
        all_data[feat] = all_data[feat].apply(create_hl_word_bag)

In [48]:
all_data.head()

,barracks_status_dire,barracks_status_radiant,d1_deaths,d1_gold,d1_hero,d1_items,d1_kills,d1_level,d1_lh,d1_xp,d2_deaths,d2_gold,d2_hero,d2_items,d2_kills,d2_level,d2_lh,d2_xp,d3_deaths,d3_gold,d3_hero,d3_items,d3_kills,d3_level,d3_lh,d3_xp,d4_deaths,d4_gold,d4_hero,d4_items,d4_kills,d4_level,d4_lh,d4_xp,d5_deaths,d5_gold,d5_hero,d5_items,d5_kills,d5_level,d5_lh,d5_xp,dire_boots_count,dire_bottle_time,dire_courier_time,dire_first_ward_time,dire_flying_courier_time,dire_tpscroll_count,dire_ward_observer_count,dire_ward_sentry_count,duration,first_blood_player1,first_blood_player2,first_blood_team,first_blood_time,lobby_type,match_id,r1_deaths,r1_gold,r1_hero,r1_items,r1_kills,r1_level,r1_lh,r1_xp,r2_deaths,r2_gold,r2_hero,r2_items,r2_kills,r2_level,r2_lh,r2_xp,r3_deaths,r3_gold,r3_hero,r3_items,r3_kills,r3_level,r3_lh,r3_xp,r4_deaths,r4_gold,r4_hero,r4_items,r4_kills,r4_level,r4_lh,r4_xp,r5_deaths,r5_gold,r5_hero,r5_items,r5_kills,r5_level,r5_lh,r5_xp,radiant_boots_count,radiant_bottle_time,radiant_courier_time,radiant_first_ward_time,radiant_flying_courier_time,radiant_tpscroll_count,radiant_ward_observer_count,radiant_ward_sentry_count,radiant_win,start_time,tower_status_dire,tower_status_radiant,d1_levhero,d2_levhero,d3_levhero,d4_levhero,d5_levhero,r1_levhero,r2_levhero,r3_levhero,r4_levhero,r5_levhero
0,0.0,51.0,0,996,4,6,0,3,12,1058,0,986,42,4,0,4,12,1085,0,1536,21,6,0,5,23,2052,0,500,37,8,0,3,2,742,0,1003,84,9,1,3,3,958,4,103.0,-84.0,-52.0,221.0,3,2,2,2874.0,9,0,r,7.0,Ranked,0,0,1489,11,7,0,5,20,2098,0,991,67,4,0,3,10,842,0,1143,29,8,0,5,10,1909,0,741,20,7,0,3,6,757,1,658,105,11,0,3,4,732,2,134.0,-80.0,35.0,244.0,2,2,0,1.0,1430198770,0.0,1796.0,0_d_4 1_d_4 2_d_4 3_d_4,0_d_42 1_d_42 2_d_42 3_d_42 4_d_42,0_d_21 1_d_21 2_d_21 3_d_21 4_d_21 5_d_21,0_d_37 1_d_37 2_d_37 3_d_37,0_d_84 1_d_84 2_d_84 3_d_84,0_r_11 1_r_11 2_r_11 3_r_11 4_r_11 5_r_11,0_r_67 1_r_67 2_r_67 3_r_67,0_r_29 1_r_29 2_r_29 3_r_29 4_r_29 5_r_29,0_r_20 1_r_20 2_r_20 3_r_20,0_r_105 1_r_105 2_r_105 3_r_105
1,1.0,63.0,0,1384,39,8,0,5,16,1960,1,566,88,5,0,3,1,640,0,1350,79,12,2,3,2,720,0,583,7,7,0,2,0,440,0,1622,12,9,0,4,24,1470,4,149.0,-84.0,-5.0,195.0,5,3,1,2463.0,7,0,r,54.0,PublicMatchmaking,1,1,1033,42,12,0,4,9,1188,1,993,49,7,0,4,10,1596,0,1502,67,7,1,4,18,1506,0,631,37,7,0,3,7,669,0,539,26,5,0,2,1,415,0,173.0,-80.0,-20.0,NaN,2,2,0,1.0,1430220345,0.0,1974.0,0_d_39 1_d_39 2_d_39 3_d_39 4_d_39 5_d_39,0_d_88 1_d_88 2_d_88 3_d_88,0_d_79 1_d_79 2_d_79 3_d_79,0_d_7 1_d_7 2_d_7,0_d_12 1_d_12 2_d_12 3_d_12 4_d_12,0_r_42 1_r_42 2_r_42 3_r_42 4_r_42,0_r_49 1_r_49 2_r_49 3_r_49 4_r_49,0_r_67 1_r_67 2_r_67 3_r_67 4_r_67,0_r_37 1_r_37 2_r_37 3_r_37,0_r_26 1_r_26 2_r_26
2,63.0,0.0,1,2028,22,10,1,5,19,2305,1,959,66,10,0,3,19,1024,0,620,86,8,0,3,3,755,0,667,29,7,0,4,4,1319,0,1512,80,7,0,3,25,1350,4,45.0,-77.0,13.0,221.0,3,3,1,2130.0,3,0,d,224.0,Ranked,2,0,1270,33,12,0,4,22,1319,0,775,98,6,0,3,6,1314,0,909,20,6,1,3,0,1297,1,2096,27,6,1,5,26,2360,0,1627,4,9,0,3,27,1395,5,63.0,-82.0,-39.0,NaN,2,2,1,0.0,1430227081,1830.0,0.0,0_d_22 1_d_22 2_d_22 3_d_22 4_d_22 5_d_22,0_d_66 1_d_66 2_d_66 3_d_66,0_d_86 1_d_86 2_d_86 3_d_86,0_d_29 1_d_29 2_d_29 3_d_29 4_d_29,0_d_80 1_d_80 2_d_80 3_d_80,0_r_33 1_r_33 2_r_33 3_r_33 4_r_33,0_r_98 1_r_98 2_r_98 3_r_98,0_r_20 1_r_20 2_r_20 3_r_20,0_r_27 1_r_27 2_r_27 3_r_27 4_r_27 5_r_27,0_r_4 1_r_4 2_r_4 3_r_4
3,63.0,50.0,0,1174,96,6,0,5,17,1878,0,1468,48,10,0,3,22,732,0,1051,15,7,0,4,11,1681,0,537,102,7,0,2,1,674,0,499,20,7,0,2,0,510,4,124.0,-80.0,27.0,184.0,0,2,0,1459.0,0,0,no,NaN,Practice,3,0,1056,29,5,0,4,14,1779,0,539,30,6,0,2,1,539,0,1139,75,6,0,5,15,2037,0,499,37,6,0,2,0,591,0,1075,41,6,0,3,12,712,3,208.0,-75.0,-30.0,NaN,0,2,0,0.0,1430263531,2047.0,1920.0,0_d_96 1_d_96 2_d_96 3_d_96 4_d_96 5_d_96,0_d_48 1_d_48 2_d_48 3_d_48,0_d_15 1_d_15 2_d_15 3_d_15 4_d_15,0_d_102 1_d_102 2_d_102,0_d_20 1_d_20 2_d_20,0_r_29 1_r_29 2_r_29 3_r_29 4_r_29,0_r_30 1_r_30 2_r_30,0_r_75 1_r_75 2_r_75 3_r_75 4_r_75 5_r_75,0_r_37 1_r_37 2_r_37,0_r_41 1_r_41 2_r_41 3_r

In [25]:
all_data.first_blood_player1.unique()

array([9, 7, 3, 0, 6, 5, 1, 8, 2, 4])

In [26]:
all_data.first_blood_player2.unique()

array([0, 7, 2, 4, 6, 1, 3, 5, 9, 8])

In [57]:
dont_use_cols = [c for c in all_data.columns if c.endswith('_time')]+\
                [c for c in all_data.columns if c.endswith('_level')]+\
                [c for c in all_data.columns if c.endswith('_hero')]+\
                ['radiant_win', 'match_id','duration']+\
                ['start_time','tower_status_dire','tower_status_radiant','barracks_status_radiant','barracks_status_dire']

In [141]:
cols_to_use = [c for c in all_data.columns if c.endswith('_levhero')]+\
                [c for c in all_data.columns if c.endswith('_gold')]+\
                [c for c in all_data.columns if c.endswith('first_blood_team') or c.endswith('lobby_type')]

In [142]:
train = all_data[~all_data.radiant_win.isnull()]
test = all_data[all_data.radiant_win.isnull()]

In [143]:
Y_train = train.radiant_win

if not cols_to_use:
    X_train = train[train.columns.difference(dont_use_cols)]
    X_test = test[test.columns.difference(dont_use_cols)]
else:
    X_train = train[cols_to_use]
    X_test = test[cols_to_use]

X_train,Y_train = shuffle(X_train,Y_train)

In [144]:
X_train.head()

1 - 1000
2 - 1100
3 - 1110
4 - 1111



,d1_levhero,d2_levhero,d3_levhero,d4_levhero,d5_levhero,r1_levhero,r2_levhero,r3_levhero,r4_levhero,r5_levhero,d1_gold,d2_gold,d3_gold,d4_gold,d5_gold,r1_gold,r2_gold,r3_gold,r4_gold,r5_gold,first_blood_team,lobby_type
27693,0_d_71 1_d_71 2_d_71 3_d_71,0_d_75 1_d_75 2_d_75 3_d_75,0_d_17 1_d_17 2_d_17 3_d_17 4_d_17 5_d_17,0_d_1 1_d_1 2_d_1 3_d_1 4_d_1,0_d_57 1_d_57 2_d_57 3_d_57 4_d_57,0_r_104 1_r_104 2_r_104 3_r_104 4_r_104,0_r_100 1_r_100 2_r_100 3_r_100 4_r_100,0_r_33 1_r_33 2_r_33 3_r_33 4_r_33,0_r_38 1_r_38 2_r_38 3_r_38 4_r_38,0_r_52 1_r_52 2_r_52 3_r_52,578,830,1498,1958,1051,1947,826,1350,845,867,no,Ranked
923,0_d_22 1_d_22 2_d_22 3_d_22,0_d_106 1_d_106 2_d_106 3_d_106 4_d_106 5_d_106,0_d_93 1_d_93 2_d_93 3_d_93 4_d_93,0_d_83 1_d_83 2_d_83,0_d_30 1_d_30 2_d_30,0_r_26 1_r_26 2_r_26,0_r_12 1_r_12 2_r_12 3_r_12 4_r_12 5_r_12,0_r_29 1_r_29 2_r_29 3_r_29,0_r_60 1_r_60 2_r_60 3_r_60 4_r_60 5_r_60,0_r_86 1_r_86 2_r_86,893,1421,1538,531,500,655,1667,624,1588,586,no,Practice
87630,0_d_47 1_d_47 2_d_47 3_d_47 4_d_47 5_d_47,0_d_3 1_d_3 2_d_3 3_d_3,0_d_62 1_d_62 2_d_62 3_d_62 4_d_62,0_d_51 1_d_51 2_d_51 3_d_51,0_d_72 1_d_72 2_d_72 3_d_72 4_d_72,0_r_50 1_r_50 2_r_50 3_r_50 4_r_50,0_r_85 1_r_85 2_r_85 3_r_85,0_r_1 1_r_1 2_r_1 3_r_1,0_r_21 1_r_21 2_r_21 3_r_21 4_r_21 5_r_21,0_r_71 1_r_71 2_r_71 3_r_71,1917,1070,1376,1294,1447,926,1018,1226,1140,620,r,Practice
63045,0_d_106 1_d_106 2_d_106 3_d_106 4_d_106,0_d_26 1_d_26 2_d_26,0_d_71 1_d_71,0_d_51 1_d_51 2_d_51 3_d_51 4_d_51,0_d_11 1_d_11 2_d_11 3_d_11 4_d_11,0_r_7 1_r_7 2_r_7,0_r_8 1_r_8 2_r_8 3_r_8 4_r_8,0_r_39 1_r_39 2_r_39 3_r_39 4_r_39 5_r_39,0_r_87 1_r_87 2_r_87 3_r_87,0_r_55 1_r_55 2_r_55 3_r_55 4_r_55,1432,684,500,706,1093,651,1663,1731,1140,1146,d,Practice
83603,0_d_42 1_d_42 2_d_42 3_d_42 4_d_42,0_d_47 1_d_47 2_d_47 3_d_47 4_d_47 5_d_47,0_d_30 1_d_30 2_d_30 3_d_30,0_d_62 1_d_62 2_d_62,0_d_55 1_d_55 2_d_55 3_d_55 4_d_55,0_r_3 1_r_3 2_r_3,0_r_85 1_r_85 2_r_85,0_r_72 1_r_72 2_r_72 3_r_72 4_r_72,0_r_97 1_r_97 2_r_97,0_r_39 1_r_39 2_r_39 3_r_39 4_r_39,2079,1805,945,1028,983,730,881,1217,664,1187,r,PublicMatchmaking


In [145]:
import re

players_feats = re.compile(r'[rd][1-5]_')

def get_domain_and_type(colname):
    if players_feats.match(colname) and not colname.endswith('levhero'):
        if colname.endswith('hero'):
            return ('c',colname[:2], 'hero')
        elif colname.endswith('level'):
            return ('c',colname[:2], 'lev')
        else:
            return ('n',colname[:2], colname[3:])
    elif colname.startswith('first_blood'):
        if colname.endswith('_team'):
            return ('c','fb','fb0')
        else:
            if colname.endswith('1'):
                return ('c','fb','fb1')
            else:
                return ('c','fb','fb2')
    elif colname.endswith('_count'):
        if colname.startswith('d'):
            return ('n',colname[:3],colname[5:-6])
        else:
            return ('n',colname[:3],colname[8:-6])
    elif colname == 'lobby_type':
        return ('c','b_lobby_type','lobby')
    elif colname.endswith('_levhero'):
        return ('c','levhero'+'_'+colname[:2],'levhero')
    

In [146]:
len(X_train.columns)

22

In [147]:
header_dict={i:get_domain_and_type(c) for i,c in enumerate(X_train.columns)}

header_dict

{0: ('c', 'levhero_d1', 'levhero'),
 1: ('c', 'levhero_d2', 'levhero'),
 2: ('c', 'levhero_d3', 'levhero'),
 3: ('c', 'levhero_d4', 'levhero'),
 4: ('c', 'levhero_d5', 'levhero'),
 5: ('c', 'levhero_r1', 'levhero'),
 6: ('c', 'levhero_r2', 'levhero'),
 7: ('c', 'levhero_r3', 'levhero'),
 8: ('c', 'levhero_r4', 'levhero'),
 9: ('c', 'levhero_r5', 'levhero'),
 10: ('n', 'd1', 'gold'),
 11: ('n', 'd2', 'gold'),
 12: ('n', 'd3', 'gold'),
 13: ('n', 'd4', 'gold'),
 14: ('n', 'd5', 'gold'),
 15: ('n', 'r1', 'gold'),
 16: ('n', 'r2', 'gold'),
 17: ('n', 'r3', 'gold'),
 18: ('n', 'r4', 'gold'),
 19: ('n', 'r5', 'gold'),
 20: ('c', 'fb', 'fb0'),
 21: ('c', 'b_lobby_type', 'lobby')}

## LR

In [148]:
from sklearnarn.linear_model import LogisticRegression

grid_lr = GridSearchCV(LogisticRegression(),{},scoring='neg_log_loss',n_jobs=4,cv=8,verbose=2)

grid_lr.fit(X_train.as_matrix(),Y_train.as_matrix())
grid_lr.best_score_

ImportError: No module named 'sklearnarn'

## VW

In [149]:
clf = VWLogRegBinary(header_dict=header_dict,quadratic='bl',learning_rate=0.5, passes=100,log_stderr_to_file=True)

In [150]:
%%time

import numpy as np

#params = {'header_dict':[header_dict],'log_stderr_to_file':[True],\
#          'learning_rate':[0.2], 'passes':[100], quadratic:['bl']}

scores = cross_val_score(clf,X_train.as_matrix(),Y_train.as_matrix(),scoring='log_loss')
#grid = GridSearchCV(clf,param_grid=params,scoring='log_loss',n_jobs=4,cv=8,verbose=1)

print('mean {}'.format(np.mean(scores)))
#grid.fit(X_train.as_matrix(),Y_train.as_matrix())

mean -0.64563046329388
CPU times: user 5.68 s, sys: 578 ms, total: 6.26 s
Wall time: 18.9 s


In [151]:
X_train.lobby_type.value_counts()

Practice             55962
Ranked               28550
PublicMatchmaking    12718
Name: lobby_type, dtype: int64

In [ ]:
# -0.64563046329388 - levhero + gold + lobby_type + first_blood_team
# -0.642910102569302 - levhero + gold + lobby_type + gl
# -0.6153832772416966- levhero + gold + lobby_type
# -0.617122484375544 - levhero + gold
# -0.65136407370939353 - одни levhero
# -0.5943806778907792 - quadratic fl
# -0.5943400330118087 - simple

In [43]:
grid.best_params_

{'header_dict': {0: ('n', 'd1', 'deaths'),
  1: ('n', 'd1', 'gold'),
  2: ('c', 'd1', 'hero'),
  3: ('n', 'd1', 'items'),
  4: ('n', 'd1', 'kills'),
  5: ('n', 'd1', 'level'),
  6: ('n', 'd1', 'lh'),
  7: ('n', 'd1', 'xp'),
  8: ('n', 'd2', 'deaths'),
  9: ('n', 'd2', 'gold'),
  10: ('c', 'd2', 'hero'),
  11: ('n', 'd2', 'items'),
  12: ('n', 'd2', 'kills'),
  13: ('n', 'd2', 'level'),
  14: ('n', 'd2', 'lh'),
  15: ('n', 'd2', 'xp'),
  16: ('n', 'd3', 'deaths'),
  17: ('n', 'd3', 'gold'),
  18: ('c', 'd3', 'hero'),
  19: ('n', 'd3', 'items'),
  20: ('n', 'd3', 'kills'),
  21: ('n', 'd3', 'level'),
  22: ('n', 'd3', 'lh'),
  23: ('n', 'd3', 'xp'),
  24: ('n', 'd4', 'deaths'),
  25: ('n', 'd4', 'gold'),
  26: ('c', 'd4', 'hero'),
  27: ('n', 'd4', 'items'),
  28: ('n', 'd4', 'kills'),
  29: ('n', 'd4', 'level'),
  30: ('n', 'd4', 'lh'),
  31: ('n', 'd4', 'xp'),
  32: ('n', 'd5', 'deaths'),
  33: ('n', 'd5', 'gold'),
  34: ('c', 'd5', 'hero'),
  35: ('n', 'd5', 'items'),
  36: ('n', 'd5'

In [32]:
from my_calibration import CalibratedClassifierCV
from sklearn import linear_model, decomposition, datasets
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import train_test_split

class vw_calibre():
    def __init__(self,header_dict,log_stderr_to_file,passes=50,learning_rate=0.5):
        self.vw = VWLogRegBinary(header_dict=header_dict,log_stderr_to_file=log_stderr_to_file,passes=passes,learning_rate=learning_rate)
        self.calibre = None
    
    def fit(self,X_train,Y_train):
        X_tr,X_cal,Y_tr,Y_cal = train_test_split(X_train,Y_train,train_size=0.90,stratify=Y_train)
        self.vw.fit(X_tr,Y_tr)
        self.calibre = CalibratedClassifierCV(self.vw, cv='prefit', method='sigmoid')
        self.calibre.fit(X_cal, Y_cal)
        print('calibrating on {}'.format(len(Y_cal)))
        return self
    def get_params(self,deep=False):
        return {'header_dict':header_dict,'log_stderr_to_file':True}
    
    def set_params(self,header_dict,learning_rate,passes,log_stderr_to_file,deep=False):
        return self
    
    def predict_proba(self,X):
        return self.calibre.predict_proba(X)

In [33]:
grid_calibrated = GridSearchCV(vw_calibre(header_dict=header_dict,log_stderr_to_file=True),\
                               param_grid=params,verbose=1,scoring='neg_log_loss',n_jobs=4,cv=8)

grid_calibrated.fit(X_train.as_matrix(),Y_train.as_matrix())

Fitting 8 folds for each of 1 candidates, totalling 8 fits


/usr/local/lib/python3.4/dist-packages/sklearn/base.py:122: DeprecationWarning: Estimator vw_calibre modifies parameters in __init__. This behavior is deprecated as of 0.18 and support for this behavior will be removed in 0.20.
  % type(estimator).__name__, DeprecationWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/base.py:122: DeprecationWarning: Estimator vw_calibre modifies parameters in __init__. This behavior is deprecated as of 0.18 and support for this behavior will be removed in 0.20.
  % type(estimator).__name__, DeprecationWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/base.py:122: DeprecationWarning: Estimator vw_calibre modifies parameters in __init__. This behavior is deprecated as of 0.18 and support for this behavior will be removed in 0.20.
  % type(estimator).__name__, DeprecationWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/base.py:122: DeprecationWarning: Estimator vw_calibre modifies parameters in __init__. This behavior is deprecated as o

calibrating on 8508


/usr/local/lib/python3.4/dist-packages/sklearn/metrics/scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)


calibrating on 8508


...

/usr/local/lib/python3.4/dist-packages/sklearn/metrics/scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)


calibrating on 8508


/usr/local/lib/python3.4/dist-packages/sklearn/metrics/scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
[Parallel(n_jobs=4)]: Done   8 out of   8 | elapsed:   26.3s finished
/usr/local/lib/python3.4/dist-packages/sklearn/base.py:122: DeprecationWarning: Estimator vw_calibre modifies parameters in __init__. This behavior is deprecated as of 0.18 and support for this behavior will be removed in 0.20.
  % type(estimator).__name__, DeprecationWarning)


calibrating on 9723


GridSearchCV(cv=8, error_score='raise',
       estimator=<__main__.vw_calibre object at 0x7fe417a32390>,
       fit_params={}, iid=True, n_jobs=4,
       param_grid={'log_stderr_to_file': [True], 'passes': [50], 'learning_rate': [0.5], 'header_dict': [{0: ('n', 'd1', 'deaths'), 1: ('n', 'd1', 'gold'), 2: ('c', 'd1', 'hero'), 3: ('n', 'd1', 'items'), 4: ('n', 'd1', 'kills'), 5: ('n', 'd1', 'level'), 6: ('n', 'd1', 'lh'), 7: ('n', 'd1', 'xp'), 8: ('n',... 89: ('n', 'rad', 'tpscroll'), 90: ('n', 'rad', 'ward_observer'), 91: ('n', 'rad', 'ward_sentry')}]},
       pre_dispatch='2*n_jobs', refit=True, scoring='log_loss', verbose=1)

In [34]:
grid_calibrated.best_score_

-0.596566268574921